# SDialog dependencies

In [1]:
import os
import json
from tqdm import tqdm

In [2]:
import sdialog
from sdialog import Dialog
from sdialog.generators import PersonaGenerator
from sdialog.personas import Persona, PersonaAgent, Doctor, Patient, Agent

/Users/yanislabrak/opt/miniconda3/envs/flask/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generate Persona

In [ ]:
persona_card_folder = "./personas"
MODEL_NAME = "gemma3:4b"

# Generate doctor persona
doctor_persona = Doctor(
    name="Dr. Smith",
    gender="male",
    age=52,
    specialty="Family Medicine"
)
generator_doctor = PersonaGenerator(doctor_persona, model=MODEL_NAME)
persona_cards = generator_doctor.generate(n=1)
persona_cards[0].to_file(f"{persona_card_folder}/persona_doctor.json")

# Generate patient persona
patient_persona = Patient(
    name="John Doe",
    gender="male",
    age=62
)
generator_patient = PersonaGenerator(patient_persona, model=MODEL_NAME)
persona_cards = generator_patient.generate(n=1)
persona_cards[0].to_file(f"{persona_card_folder}/persona_patient.json")


# Load persona

In [ ]:
# Load personas
persona_doctor = Persona.from_file("./personas/persona_doctor.json")
persona_patient = Persona.from_file("./personas/persona_patient.json")

# Create agents
agent1 = PersonaAgent(persona=persona_doctor, model="gemma3:4b", name="DOCTOR")
agent2 = PersonaAgent(persona=persona_patient, model="gemma3:4b", name="PATIENT")

In [10]:
save_all = False
GENERATE_PERSONA = True
FORCE_DIALOG_GENERATION = False

os.makedirs("./outputs", exist_ok=True)

In [ ]:
if FORCE_DIALOG_GENERATION:

    dialog = agent1.talk_with(agent2)
    dialog.to_file("dialog_demo.json")

else:
    dialog = Dialog.from_file("dialog_demo.json")

dialog.print()

# Audio dependencies

In [ ]:
from sdialog.audio.voice_database import DummyVoiceDatabase
from sdialog.audio.tts_engine import KokoroTTS # ChatterboxTTS, XttsTTS
from sdialog.audio.audio_events_enricher import AudioEventsEnricher
from sdialog.audio import dialog_to_audio, to_wav, generate_utterances_audios
from sdialog.audio.evaluation import compute_evaluation_utterances, compute_evaluation_audio

# Instantiating voices database

In [ ]:
dummy_voice_database = DummyVoiceDatabase()
print("Instantiating voice database...")

# Instantiating TTS pipeline

In [ ]:
tts_pipeline = KokoroTTS()
# tts_pipeline = ChatterboxTTS()
# tts_pipeline = XttsTTS()
print("Instantiating TTS pipeline...")

In [ ]:
audio_res = dialog_to_audio(dialog, voice_database=dummy_voice_database, tts_pipeline=tts_pipeline)

In [16]:
if save_all:

    # Saving individual utterances audios
    os.makedirs("./outputs/utterances", exist_ok=True)
    for idx, (utterance, speaker) in enumerate(audio_res["utterances_audios"]):
        to_wav(utterance, f"./outputs/utterances/{idx}_{speaker}_utterance.wav")

    # Saving audio step 1: no room accoustics and audios events
    to_wav(audio_res["audio"], "./outputs/dialog_audio_step1.wav")

In [ ]:
# Enriching the dialog with audio events and generate the timeline of audio events and utterances
enricher = AudioEventsEnricher()
timeline = enricher.extract_events(dialog, audio_res["utterances_audios"])
timeline.print()

# Drawing the timeline
timeline.draw("./outputs/timeline.png")

In [ ]:
compute_evaluation_utterances(audio_res["utterances_audios"], dialog)
compute_evaluation_audio(audio_res["audio"], dialog)